# Dncnn with Assumed Density Filtering 

## Library import

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from keras.layers import  Input,Conv2D,Activation
from glob import glob
import time as time
from six.moves import xrange
import os
import random
from PIL import Image
import cv2 
import scipy.misc as misc
import random
import warnings
warnings.filterwarnings('ignore')

## Dataset preprocessor 
### Build data pipline for the network

In [168]:
class dataset_preprocessor(object):
    def __init__(self,sess,clean_filepath,noisy_filepath,batch_size,image_shape):
        self.clean_filepath = glob(clean_filepath)
        self.noisy_filepath = glob(noisy_filepath)
        self.batch_size = batch_size
        self.image_shape = image_shape ### input should be a 2-element list like:[iamge_width,image_height]
        self.sess = sess        
        self.train_list = self.clean_filepath + self.noisy_filepath
        
        def random_int_list(start, stop, length):
            start, stop = (int(start), int(stop)) if start <= stop else (int(stop), int(start))
            length = int(abs(length)) if length else 0
            random_list = []
            for i in range(length):
                random_list.append(random.randint(start, stop))
            return random_list
        
        def process_image(image):
            image_string = tf.read_file(image)
            image_decoded = tf.image.decode_image(image_string)
            image_decoded.set_shape([None, None, None])
            image_resized = tf.image.resize_images(image_decoded, self.image_shape)  
            image_converted = tf.cast(image_resized, tf.float32)
            return image_converted
        
        def shuffle_dataset(list1,list2):
            res = []
            for i in range(400):
                seed = random.choice(list2)
                list2.remove(seed)
                res.append(list1[seed])
                res.append(list1[seed + 400])
            return res

            
        shuffle_list = random_int_list(0,399,400)       
        self.shuffle_pip = shuffle_dataset(self.train_list,shuffle_list)   
        dataset = tf.data.Dataset.from_tensor_slices(self.shuffle_pip)
        dataset = dataset.map(process_image,num_parallel_calls = 1)
        dataset = dataset.batch(self.batch_size * 2)
        dataset = dataset.repeat()
        data_iterator = dataset.make_one_shot_iterator() ## format of the pipline likes [clean1,noisy1,clean2,noisy2...]
        self.next_data = data_iterator.get_next()

    def get_dataset(self):
        return self.sess.run(self.next_data)
    

### Build test data pipline for the network

In [169]:
class testset_preprocessor(object):
    def __init__(self,sess,filepath,batch_size,image_shape):
        self.filepath = glob(filepath)
        self.batch_size = batch_size
        self.image_shape = image_shape ### input should be a 2-element list like:[iamge_width,image_height]
        self.sess = sess        
        
        def process_image(image):
            image_string = tf.read_file(image)
            image_decoded = tf.image.decode_image(image_string)
            image_decoded.set_shape([None, None, None])
            image_resized = tf.image.resize_images(image_decoded, self.image_shape)  
            image_converted = tf.cast(image_resized, tf.float32)
            return image_converted

                   
        dataset = tf.data.Dataset.from_tensor_slices(self.filepath)
        dataset = dataset.map(process_image,num_parallel_calls = 1)
        dataset = dataset.shuffle(2)
        dataset = dataset.batch(self.batch_size)
        dataset = dataset.repeat()
        data_iterator = dataset.make_one_shot_iterator()
        self.next_data = data_iterator.get_next()

    def get_dataset(self):
        return self.sess.run(self.next_data)

### 1.Normal version DNCNN(with Batch-Normalization)

In [170]:
class DNCNN(object):
    def __init__(self,img_channel,batch_size,image_height,image_width,
                 stride,padding,epoch_number,filters_number,
                 kernel_size,noise_sigma,):
        self.sess = tf.Session()
        self.img_channel = img_channel
        self.batch_size = batch_size
        self.image_height = image_height
        self.image_width = image_width
        self.stride = stride
        self.padding = padding
        self.epoch_number = epoch_number
        self.filters_number = filters_number
        self.kernel_size = kernel_size
        self.noise_sigma = noise_sigma
        self.pipline = dataset_preprocessor(sess = self.sess,clean_filepath='data_set/train_clean/*.png',
                               noisy_filepath='data_set/train_noisy/*.png',batch_size=self.batch_size,image_shape=[self.image_width,self.image_height])
        self.model_build()
        
    ## model build inside forward propergation and loss 
    ## Graph
    def model_build(self):
        
        self.noisy_image = tf.placeholder(tf.float32,[None,self.image_height,self.image_width,self.img_channel],name = 'noisy_image')
        self.clean_image = tf.placeholder(tf.float32,[None,self.image_height,self.image_width,self.img_channel],name = 'clean_image')
        self.train_phase = tf.placeholder(tf.bool)
        '''
        For other general image denoising tasks, we adopt a larger receptive field and set the depth to be 20      
        ''' 
        ### input layer
        with tf.variable_scope('nn_layer1'):
            output = tf.layers.conv2d(self.noisy_image, 
                                      filters = self.filters_number,
                                     kernel_size = self.kernel_size,
                                     padding = self.padding,
                                     strides = self.stride,
                                     activation=tf.nn.relu)
        
        # layer 2 to 19
        for i in range(20-2):
            with tf.variable_scope('nn_layer' + str(i+2)):
                output = tf.layers.conv2d(output,filters = self.filters_number,
                                     kernel_size = self.kernel_size,
                                     padding = self.padding,
                                     strides = self.stride                                 
                                     )
                output = tf.nn.relu(tf.layers.batch_normalization(output, training=self.train_phase)) 
        
        # layer 20
        with tf.variable_scope('nn_layer20'):
            noisy_learned = tf.layers.conv2d(output,filters = self.filters_number,
                                     kernel_size = self.kernel_size,
                                     padding = self.padding,
                                    strides = self.stride
                                     )    
         ### input - noisy image(we gain it from data set)
        self.pure_noisy_image = self.noisy_image - self.clean_image
        self.denoised_image = self.noisy_image - noisy_learned
        self.psnr = tf.image.psnr(self.denoised_image, self.noisy_image, max_val=255.0)[0]
        self.loss = tf.reduce_mean(tf.keras.losses.MSE(self.pure_noisy_image,noisy_learned))
#         self.loss = (1.0/self.batch_size) * tf.nn.l2_loss(self.pure_noisy_image - noisy_learned)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001,
                                                beta1=0.9,
                                                beta2=0.999,
                                                epsilon=1e-08,
                                                use_locking=False,
                                                name='Adam')
        self.train_step = self.optimizer.minimize(self.loss)
        self.global_init = tf.global_variables_initializer()  ### initializer the variable in every epoch
        print("[*] initialing the model")
        self.sess.run(self.global_init)
        print("[*] Saving initialized model")
        self.saver = tf.train.Saver()
        self.saver.save(self.sess,"./save_model/DNCNN_with_BN/DNCNN")
        print("[*] Finishing saving and initialing")
        
        
        ''' 
        Formally, the averaged mean squared error between the desired residual images and estimated ones from 
        noisy input can be adopted as the loss function to learn the trainable parameters Θ in DnCNN.
        ient-based optimization algorithms and network architecture. Note that two gradient-based optimization algorithms are adopted: 
        one is the stochastic gradient descent algorithm with momentum (i.e., SGD) and the other one is the Adam algorithm [30].
        '''
    def train(self):        
        ## collect the loss and psnr in certain train_step
        tf.summary.scalar('loss', self.loss)
        tf.summary.scalar('psnr', self.psnr)
        writer = tf.summary.FileWriter('./evaluate', self.sess.graph)
        merged = tf.summary.merge_all()       
        
        ## start training model
        print("[*] Start training model")
        for epoch in range(self.epoch_number): ## train all data in epoch_number times            
            for iteration in range(40): ## each epoch_number has  filepath/self.batch_ize dataset                
                cleaned_batch = np.zeros((self.batch_size,self.image_height,self.image_width,self.img_channel),dtype = 'float32')
                noised_batch  = np.zeros((self.batch_size,self.image_height,self.image_width,self.img_channel),dtype = 'float32')
                train_set = self.pipline.get_dataset()
                ind = 0
                for index in range(0,2*self.batch_size,2):
                    cleaned_batch[ind,:,:,:] = train_set[index] 
                    noised_batch[ind,:,:,:] = train_set[index+1]
                    ind = ind + 1
                [_,loss,summary] = self.sess.run([self.train_step,self.loss,merged],feed_dict = {self.clean_image:cleaned_batch, self.noisy_image:noised_batch,self.train_phase: True})
#                                                                                             self.noisy_image:noised_batch,
#                                                                                                self.train_phase: True})
                writer.add_summary(summary, iteration)
                if((iteration%10) ==0):
                    [psnr,loss] = self.sess.run([self.psnr,self.loss],
                                            feed_dict = {self.clean_image:cleaned_batch,
                                            self.noisy_image:noised_batch,self.train_phase : False})
                    print("[*] Saving model")
                    self.saver.save(self.sess,"save_model/DnCNN_with_BN",global_step = epoch+1)
                    print("[*] Training.... : Epoch: "+ str(epoch) + ", Step: " + str(iteration) + ", PSNR: " + str(psnr) + ", LOSS: " + str(loss))
                if(epoch+1 == self.epoch_number):
                    self.saver.save(self.sess,"save_model/DnCNN_with_BN",global_step = epoch+1)
                    [psnr,loss,denoised_image] = self.sess.run([self.psnr,self.loss,self.denoised_image],
                                                                  feed_dict = {self.clean_image:cleaned_batch,
                                                                self.noisy_image:noised_batch,self.train_phase : False})
                    print("[*] Final loss and psnr: LOSS: " + str(loss) + " PSNR: " + str(psnr))
                    compared = np.concatenate((cleaned_batch[0, :, :, 0], noised_batch[0, :, :, 0], denoised_image[0, :, :, 0]), 1)
                    Image.fromarray(np.uint8(compared)).save(str(epoch)+"_"+str(iteration)+".png")
        
        
        print("[*] Finish Training")

    def load_pretrained_model(self,ckpt_dir,meta_dir):
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt:
            saver=tf.train.import_meta_graph(meta_path)
            saver.restore(self.sess, ckpt_path)
            return saver
        else:
            print("Can't find pretrained model")
    
    
    def test(self,ckpt_dir,meta_dir,test_filepath,times_of_testing,noisy_sigma): ## input times_of_testing must be a dict
                                                                                ## follow such struct {'num_of_images':_ ,'num_of_test_batch':_ }
        savevr = load_pretrained_model(ckpt_dir,meta_dir)
        assert saver != None , print("Error message: No model find")
        test_pip = testset_preprocessor(sess = self.sess,filepath = test_filepath,
                                        batch_size = times_of_testing['num_of_images'],
                                        image_shape = [self.image_width,self.image_height])
        path = "/test_DNCNN_with_BN"
        for i in range(times_of_testing['num_of_test_batch']):
            clean_img = test_pip.get_dataset()
            noised_img = cleaned_img + np.random.normal(0, noise_sigma, cleaned_img.shape)
            [denoised_img] = self.sess.run([self.denoised_img], feed_dict={self.clean_img: clean_img, self.noised_img: noised_img, self.train_phase: False})
            for j in len(clean_img):
                compared = np.concatenate((cleaned_img[0, :, :, 0], noised_img[0, :, :, 0], denoised_img[0, :, :, 0]), 1)
                cv2.imwrite(os.path.join(path , "test_DNCNN_with_BN_" + str(j) + ".png"), Image.fromarray(np.uint8(compared)))
                

In [172]:
tf.reset_default_graph()
dncnn = DNCNN(img_channel = 3, batch_size = 10,image_height = 180,image_width = 180,stride = (1,1),padding = 'same', epoch_number = 5,filters_number = 3,kernel_size = (3,3),noise_sigma = 25)
dncnn.train()

### 2.Normal DNCNN without Batch-Normalization

In [ ]:
class DNCNN(object):
    def __init__(self,img_channel,batch_size,image_height,image_width,
                 stride,padding,epoch_number,filters_number,
                 kernel_size,noise_sigma,):
        self.sess = tf.Session()
        self.img_channel = img_channel
        self.batch_size = batch_size
        self.image_height = image_height
        self.image_width = image_width
        self.stride = stride
        self.padding = padding
        self.epoch_number = epoch_number
        self.filters_number = filters_number
        self.kernel_size = kernel_size
        self.noise_sigma = noise_sigma
        self.pipline = dataset_preprocessor(sess = self.sess,clean_filepath='data_set/train_clean/*.png',
                               noisy_filepath='data_set/train_noisy/*.png',batch_size=self.batch_size,image_shape=[self.image_width,self.image_height])
        self.model_build()
        
    ## model build inside forward propergation and loss 
    ## Graph
    def model_build(self):
        
        self.noisy_image = tf.placeholder(tf.float32,[None,self.image_height,self.image_width,self.img_channel],name = 'noisy_image')
        self.clean_image = tf.placeholder(tf.float32,[None,self.image_height,self.image_width,self.img_channel],name = 'clean_image')
        self.train_phase = tf.placeholder(tf.bool)
        '''
        For other general image denoising tasks, we adopt a larger receptive field and set the depth to be 20      
        ''' 
        ### input layer
        with tf.variable_scope('nn_layer1'):
            output = tf.layers.conv2d(self.noisy_image, 
                                      filters = self.filters_number,
                                     kernel_size = self.kernel_size,
                                     padding = self.padding,
                                     strides = self.stride,
                                     activation=tf.nn.relu)
        
        # layer 2 to 19
        for i in range(20-2):
            with tf.variable_scope('nn_layer' + str(i+2)):
                output = tf.layers.conv2d(output,filters = self.filters_number,
                                     kernel_size = self.kernel_size,
                                     padding = self.padding,
                                     strides = self.stride,
                                    activation=tf.nn.relu
                                     )
                
        
        # layer 20
        with tf.variable_scope('nn_layer20'):
            noisy_learned = tf.layers.conv2d(output,filters = self.filters_number,
                                     kernel_size = self.kernel_size,
                                     padding = self.padding,
                                    strides = self.stride
                                     )    
         ### input - noisy image(we gain it from data set)
        self.pure_noisy_image = self.noisy_image - self.clean_image
        self.denoised_image = self.noisy_image - noisy_learned
        self.psnr = tf.image.psnr(self.denoised_image, self.noisy_image, max_val=255.0)[0]
        self.loss = tf.reduce_mean(tf.keras.losses.MSE(self.pure_noisy_image,noisy_learned))
#         self.loss = (1.0/self.batch_size) * tf.nn.l2_loss(self.pure_noisy_image - noisy_learned)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001,
                                                beta1=0.9,
                                                beta2=0.999,
                                                epsilon=1e-08,
                                                use_locking=False,
                                                name='Adam')
        self.train_step = self.optimizer.minimize(self.loss)
        self.global_init = tf.global_variables_initializer()  ### initializer the variable in every epoch
        print("[*] initialing the model")
        self.sess.run(self.global_init)
        print("[*] Saving initialized model")
        self.saver = tf.train.Saver(max_to_keep = 10)
        self.saver.save(self.sess,"./save_model/DNCNN_with_BN/DNCNN")
        print("[*] Finishing saving and initialing")
        
        
        ''' 
        Formally, the averaged mean squared error between the desired residual images and estimated ones from 
        noisy input can be adopted as the loss function to learn the trainable parameters Θ in DnCNN.
        ient-based optimization algorithms and network architecture. Note that two gradient-based optimization algorithms are adopted: 
        one is the stochastic gradient descent algorithm with momentum (i.e., SGD) and the other one is the Adam algorithm [30].
        '''
    def train(self):        
        ## collect the loss and psnr in certain train_step
        tf.summary.scalar('loss', self.loss)
        tf.summary.scalar('psnr', self.psnr)
        writer = tf.summary.FileWriter('./evaluate', self.sess.graph)
        merged = tf.summary.merge_all()       
        
        ## start training model
        print("[*] Start training model")
        for epoch in range(self.epoch_number): ## train all data in epoch_number times
            
            for iteration in range(40): ## each epoch_number has  filepath/self.batch_ize dataset                
                cleaned_batch = np.zeros((self.batch_size,self.image_height,self.image_width,self.img_channel),dtype = 'float32')
                noised_batch  = np.zeros((self.batch_size,self.image_height,self.image_width,self.img_channel),dtype = 'float32')
                train_set = self.pipline.get_dataset()
                ind = 0
                for index in range(0,2*self.batch_size,2):
                    cleaned_batch[ind,:,:,:] = train_set[index] 
                    noised_batch[ind,:,:,:] = train_set[index+1]
                    ind = ind + 1
                [_,loss,summary] = self.sess.run([self.train_step,self.loss,merged],feed_dict = {self.clean_image:cleaned_batch, self.noisy_image:noised_batch,self.train_phase: True})
#                                                                                             self.noisy_image:noised_batch,
#                                                                                                self.train_phase: True})
                writer.add_summary(summary, iteration)
                if((epoch%10) ==0):
                    [psnr,loss] = self.sess.run([self.psnr,self.loss],
                                            feed_dict = {self.clean_image:cleaned_batch,
                                            self.noisy_image:noised_batch,self.train_phase : False})
                    print("[*] Training.... : Epoch: "+ str(epoch) + ", Step: " + str(iteration) + ", PSNR: " + str(psnr) + ", LOSS: " + str(loss))
                if(epoch+1 == self.epoch_number):
                    print("[*] Saving model")
                    self.saver.save(self.sess,"save_model/DnCNN_with_BN",global_step = epoch+1)
                    [psnr,loss,denoised_image] = self.sess.run([self.psnr,self.loss,self.denoised_image],
                                                              feed_dict = {self.clean_image:cleaned_batch,
                                                                          self.noisy_image:noised_batch,self.train_phase : False})
                    compared = np.concatenate((cleaned_batch[0, :, :, 0], noised_batch[0, :, :, 0], denoised_image[0, :, :, 0]), 1)
                    Image.fromarray(np.uint8(compared)).save(str(epoch)+"_"+str(iteration)+".png")
        
        
        print("[*] Finish Training")

    def load_pretrained_model(self,ckpt_dir,meta_dir):
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt:
            saver=tf.train.import_meta_graph(meta_path)
            saver.restore(self.sess, ckpt_path)
            return saver
        else:
            print("Can't find pretrained model")
    
    
    def test(self,ckpt_dir,meta_dir,test_filepath,times_of_testing,noisy_sigma): ## input times_of_testing must be a dict
                                                                                ## follow such struct {'num_of_images':_ ,'num_of_test_batch':_ }
        savevr = load_pretrained_model(ckpt_dir,meta_dir)
        assert saver != None , print("Error message: No model find")
        test_pip = testset_preprocessor(sess = self.sess,filepath = test_filepath,
                                        batch_size = times_of_testing['num_of_images'],
                                        image_shape = [self.image_width,self.image_height])
        path = "/test_DNCNN_with_BN"
        for i in range(times_of_testing['num_of_test_batch']):
            clean_img = test_pip.get_dataset()
            noised_img = cleaned_img + np.random.normal(0, noise_sigma, cleaned_img.shape)
            [denoised_img] = self.sess.run([self.denoised_img], feed_dict={self.clean_img: clean_img, self.noised_img: noised_img, self.train_phase: False})
            for j in len(clean_img):
                compared = np.concatenate((cleaned_img[0, :, :, 0], noised_img[0, :, :, 0], denoised_img[0, :, :, 0]), 1)
                cv2.imwrite(os.path.join(path , "test_DNCNN_with_BN_" + str(j) + ".png"), Image.fromarray(np.uint8(compared)))
                

In [ ]:
tf.reset_default_graph()
dncnn = DNCNN(img_channel = 1, batch_size = 40,image_height = 180,image_width = 180,stride = (1,1),padding = 'same', epoch_number = 50,filters_number = 3,kernel_size = (3,3),noise_sigma = 25)
dncnn.train()

### 3.Normal DNCNN plus ADF without Batch-Normalisation

In [ ]:
import re
import os, datetime
from glob import glob
import numpy as np
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract
from keras.layers import Input, Lambda
from keras.models import Model, load_model
from keras.optimizers import Adam
import keras.backend as K
import cv2
from tensorflow import distributions as dist
import logging
import warnings
warnings.filterwarnings('ignore')
from numbers import Number

### ADF framework Activation

In [ ]:
def normcdf(value, mu=0.0, stddev=1.0):
    sinv = (1.0 / stddev) if isinstance(stddev, Number) else stddev.reciprocal()
    return 0.5 * (1.0 + tf.math.erf((value - mu) * sinv / np.sqrt(2.0)))

def _normal_log_pdf(value, mu, stddev):
    var = (stddev ** 2)
    log_scale = np.log(stddev) if isinstance(stddev, Number) else torch.log(stddev)
    return -((value - mu) ** 2) / (2.0*var) - log_scale - np.log(np.sqrt(2.0*np.pi))


def normpdf(value, mu=0.0, stddev=1.0):
    return tf.math.exp(_normal_log_pdf(value, mu, stddev))


def _lpdn_relu_activation(m, v):
#     v = tf.maximum(v, 0.0001)
#     v = tf.math.exp(v)
#     s = v ** 0.5
#     u_out = u * normal.cdf(u / s) + s * normal.prob(u / s)
#     v_out = (u + v) * normal.cdf(u / s) + (u * s) * normal.prob(u / s) - u_out ** 2
#     v_out = tf.math.log(v_out)
    
    stddev = tf.math.sqrt(v)
    div = m/stddev
    pdf = normpdf(div)
    cdf = normcdf(div)
    out_m = m*cdf + stddev*pdf
    out_v = (m ** 2 + v)*cdf + m*stddev*pdf - out_m ** 2
    out_v = tf.math.log(out_v)
    return out_m, out_v


class LPActivation(Activation): ### forward
    def __init__(self, activation, **kwargs):
        if isinstance(activation, str):
            self.activation_name = activation
        else:
            self.activation_name = activation.__name__
        super(LPActivation, self).__init__(activation, **kwargs)

    def call(self, inputs):
        u = inputs[..., 0]
        v = inputs[..., 1] ## log
        u, v = _lpdn_relu_activation(u, v)
        v = tf.math.log(v)
        return tf.stack([u,v],-1)

### ADF framework Conv2D

In [ ]:
class LPConv2D(Conv2D):
    """
    Propagate distributions over a probabilistic Conv2D layer
    """

    def __init__(self, filters, kernel_size, **kwargs):
        super(LPConv2D, self).__init__(filters, kernel_size, **kwargs)

    def build(self, input_shape):
        return super(LPConv2D, self).build(input_shape[:-1])

    def compute_output_shape(self, input_shape):
        original_output_shape = super(LPConv2D, self).compute_output_shape(input_shape[:-1])
        return original_output_shape + (2,)

    def assert_input_compatibility(self, inputs):
        return super(LPConv2D, self).assert_input_compatibility(inputs[..., 0])

    def _conv2d(self, input, kernel):
        return K.conv2d(input, kernel, self.strides, self.padding, self.data_format, self.dilation_rate)

    def call(self, inputs): ### forward
        u = inputs[..., 0]
        v = inputs[..., 1]
        
        v = tf.math.exp(v)
        
        u = self._conv2d(u, self.kernel)
        v = self._conv2d(v, self.kernel ** 2)
        
        v = tf.math.log(v)
        if self.use_bias:
            u = K.bias_add(
                u,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            u, v = _lpdn_relu_activation(u, v)      
            v = tf.math.log(v)
        return tf.stack([u,v],-1)


In [ ]:
def data_aug(img, mode=0):

    if mode == 0:
        return img
    elif mode == 1:
        return np.flipud(img)
    elif mode == 2:
        return np.rot90(img)
    elif mode == 3:
        return np.flipud(np.rot90(img))
    elif mode == 4:
        return np.rot90(img, k=2)
    elif mode == 5:
        return np.flipud(np.rot90(img, k=2))
    elif mode == 6:
        return np.rot90(img, k=3)
    elif mode == 7:
        return np.flipud(np.rot90(img, k=3))


def datagenerator(data_dir = 'data_set/train_clean/*.png',verbose=False):
    
    file_list = glob(data_dir)  
    # initrialize
    data = []
    # generate patches
    for i in range(len(file_list)):
        img = cv2.imread(file_list[i])
        img = img.resize((180,180))
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        final_img = np.zeros((180,180,1,2))
        final_img[...,0]=img
        final_img[...,1]=img
        data.append(final_img)
    data = np.array(data,dtype = 'float32')
    print('^_^-training data finished-^_^')
    return data

def train_datagen(epoch_iter=20,epoch_num=1,batch_size=10,data_dir='data_set/train_clean/*.png'):
    while(True):
        n_count = 0
        if n_count == 0:
            xs = datagenerator(data_dir)
            xs = xs.astype('float32')/255.0
            indices = list(range(xs.shape[0]))
            n_count = 1
        for _ in range(epoch_num):
            np.random.shuffle(indices)    # shuffle
            for i in range(0, len(indices), batch_size):
                batch_x = xs[indices[i:i+batch_size]]
#                 print(batch_x.shape)
                noise =  np.random.normal(0, 25, batch_x.shape)    # noise
                batch_y = batch_x + noise 
                yield batch_y, batch_x


In [ ]:
def DnCNN(depth, filters, image_channels):
    layer_count = 0
    inpt = Input(shape=(None, None, image_channels), name='input' + str(layer_count))
    # 1st layer, Conv+relu
    
    layer_count += 1
    x = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), kernel_initializer='Orthogonal', padding='same',
               name='conv' + str(layer_count))(inpt)
    layer_count += 1
    x = Activation('relu', name='relu' + str(layer_count))(x)
    # depth-2 layers, Conv+relu
    for i in range(depth - 2):
        layer_count += 1
        x = Conv2D(filters=filters, kernel_size=(3, 3), strides=(1, 1), kernel_initializer='Orthogonal', padding='same',
                   use_bias=False, name='conv' + str(layer_count))(x)
        layer_count += 1
        x = Activation('relu', name='relu' + str(layer_count))(x)
    # last layer, Conv
    layer_count += 1
    x = Conv2D(filters=image_channels, kernel_size=(3, 3), strides=(1, 1), kernel_initializer='Orthogonal',
               padding='same', use_bias=False, name='conv' + str(layer_count))(x)
    layer_count += 1
#     x[...,0] = x1[...,0] - x[...,0]
#    
    model = Model(inputs=inpt, outputs= x)
    return model

def convert(keras_model, input_shape=None):
    # Create an equivalent probabilistic model.
    if input_shape is None:
        input_shape = keras_model.layers[0].input_shape[1:] + (2,)
        logging.info("Inferred input shape: " + str(input_shape))

    lp_input = Input(shape=input_shape)
    y = lp_input
    for li, l in enumerate(keras_model.layers):
        if isinstance(l, Conv2D):
            y = LPConv2D(l.filters, l.kernel_size, padding=l.padding, activation=l.activation, name=l.name)(y)
        elif isinstance(l, Activation):
            y = LPActivation(l.activation, name=l.name)(y)
    model = Model(inputs=lp_input, outputs=y)
    return model

In [ ]:
def ADF_loss(y_true, y_pred):
    mean = tf.maximum(0.0001,y_pred[...,0])
    print(mean[0])
    variance = tf.maximum(0.0001,tf.math.exp(y_pred[...,1]))
    #print(variance.shape)
    eps = 1e-5
    temp = 1/(variance + eps)
    output = K.sum(0.5*temp*(y_true[...,0] - mean)**2 + 0.5*tf.math.log(variance + eps))
    print(output)
    return output

In [ ]:
model = DnCNN(depth=17,filters=64,image_channels=1)
model_lpdn = convert(model)
# print(model_lpdn.layers[16].output_shape)
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_lpdn.compile(optimizer=Adam(0.001), loss=['mse'])
history = model_lpdn.fit_generator(train_datagen(batch_size=5),
                steps_per_epoch=20, epochs=5, verbose=1)

### Above three model all training on the same data set and all implement data augmentation just based on fliping 

## Evaulate each model

(180, 180)